In [1]:
import sys
import os
sys.path.insert(0, os.path.abspath('../src'))


import numpy as np
from tqdm.notebook import trange
from tqdm.notebook import tqdm
from docplex.mp.model import Model

import model
import model_parameters as MP

## Mathematical Model

#### Sets
- C	Set of campaigns.
- U	Set of customers.
- H	Set of channels
- D	Set of planning days.
- I	Set of quota categories.
- P	Set of priority categories.


In [2]:
print(f"number of campaigns {MP.C}")
print(f"number of customers {MP.U}")
print(f"number of channels {MP.H}")
print(f"number of planning days {MP.D}")
print(f"number of quota categories {MP.I}")
print(f"number of priority categories {MP.P}")

number of campaigns 10
number of customers 10000
number of channels 3
number of planning days 7
number of quota categories 3
number of priority categories 10


#### Parameters

##### - eligibility
$$
e_{cu}\left\{\begin{array}\\
        1 & \mbox{if }  customer\ u\ is\ eligible\ for\ campaign\ c\\
        0 & \mbox{otherwise } \\
    \end{array}
\right.
$$

In [3]:
MP.e_cu

array([[0, 0, 0, ..., 0, 1, 1],
       [1, 0, 0, ..., 0, 0, 1],
       [0, 1, 0, ..., 1, 1, 1],
       ...,
       [1, 0, 1, ..., 1, 1, 0],
       [1, 1, 0, ..., 1, 1, 0],
       [0, 1, 1, ..., 1, 0, 0]])

##### - quota categories
$$
q_{ic}\left\{\begin{array}\\
        1 & \mbox{if }  campaign\ c\ is\ a\ i^{th} type\ quota\ category\ campaign\ \\
        0 & \mbox{otherwise } \\
    \end{array}
\right.
$$

In [4]:
MP.q_ic

array([[0, 1, 1, 0, 1, 0, 0, 0, 0, 0],
       [0, 1, 0, 1, 1, 0, 0, 0, 0, 1],
       [0, 1, 0, 1, 1, 1, 0, 1, 0, 0]])

##### - priority categories
$$r_{cp}: Priority\ value\ of\ campaign\ c\ regarding\ priority\ type\ p\$$

In [5]:
MP.rp_c

array([69, 43, 43, 87, 92, 78, 43, 43, 19, 43])

##### - blokage
$$b: Communication\ limit\ per\ person\ for\ the\ whole\ period\$$

In [6]:
MP.b

7

##### - daily blokage
$$k: Communication\ limit\ per\ person\ at\ each\ day\$$

In [7]:
MP.k

3

##### - campaign blockage
$$l_c: Communication\ limit\ per\ person\ for\ campaign\ c\$$

In [8]:
MP.l_c

array([4, 4, 3, 4, 4, 4, 4, 3, 3, 2])

##### - quota limitations daily/weekly
$$
m_i: Communication\ limit\ per\ person\ for\ i^{th}\ category\
$$
$$
n_i: Communication\ limit\ per\ person\ for\ i^{th}\ category\ each\ day\
$$

In [9]:
(MP.m_i, MP.n_i)

(array([4, 5, 4]), array([1, 2, 2]))

#### - capacity for channel
$$
t_{h,d}: Capacity\ for\ channel\ h\ at\ day\ d.\
$$

In [10]:
MP.t_hd

array([[5000., 6000., 5000., 6000., 6000., 5000., 5000.],
       [5000., 6000., 7000., 6000., 7000., 6000., 6000.],
       [6000., 7000., 7000., 7000., 5000., 7000., 6000.]])

### Model

In [11]:
mdl = Model(name='Campaign Optimization')

#### Variables
$$
X_{cuhd}\left\{\begin{array}\\
        1 & \mbox{if } Campaign\ c\ will\ be\ sent\ to\ customer\ u\ through\ Channel\ h\ at\ Day\ d \\
        0 & \mbox{otherwise } \\
    \end{array}
\right.
$$

In [12]:
X_cuhd = {(c,u,h,d): mdl.continuous_var(lb=0, ub=1, name=f"X_c:{c}_u:{u}_h:{h}_d:{d}")
            for c in range(0,MP.C)
            for u in range(0,MP.U) 
            for h in range(0,MP.H)
            for d in range(0,MP.D)}

## Maximize
$$\sum_{p \in P}\sum_{c \in C}\sum_{u \in U}\sum_{h \in H}\sum_{d \in D}\,X_{cuhd}\ r_{cp}$$

##### Binary variable (10)
$$
X_{cuhd} \in \{ 1,0 \},\hspace{35pt} \forall c \in C ,\forall u \in U,\forall d \in D, \forall h \in H
$$

In [13]:
maximize = mdl.maximize(mdl.sum([X_cuhd[(c,u,h,d)] * MP.rp_c[c]
                  for c in range(0,MP.C)
                  for u in range(0,MP.U) 
                  for h in range(0,MP.H) 
                  for d in range(0,MP.D)]))

## subject to

##### - eligibility (2)

$$
X_{cuhd}  \leq e_{cu},\hspace{35pt} \forall c \in C,\forall u \in U,\forall h \in H,\forall d \in D
$$

In [14]:
eligibilitiy = mdl.add_constraints(
    (X_cuhd[(c,u,h,d)] <= MP.e_cu[c,u]
    for c in range(0,MP.C)
    for u in range(0,MP.U) 
    for h in range(0,MP.H) 
    for d in range(0,MP.D))
)

##### - use one channel (3)
$$
\sum_{h}X_{cuhd} \le 1,\hspace{35pt} \forall c \in C \, \forall u \in U,\forall d \in D
$$

In [15]:
one_channel = mdl.add_constraints(
    ((mdl.sum(X_cuhd[(c,u,h,d)] for h in range(0,MP.H)) <= 1)
    for c in range(0,MP.C)
    for u in range(0,MP.U) 
    for d in range(0,MP.D))
)

##### - weekly communication limitation (4)
$$
\sum_{h \in H}\sum_{c \in C}\sum_{d \in D} X_{cuhd}\le b,\hspace{35pt} \forall u \in U
$$

In [16]:
weekly_communication = mdl.add_constraints(
    (
        (mdl.sum(X_cuhd[(c,u,h,d)] 
               for d in range(0,MP.D) 
               for c in range(0,MP.C) 
               for h in range(0,MP.H)) <= MP.b)
        for u in range(0,MP.U)))

##### - daily communication limitation (5)
$$
\sum_{h \in H}\sum_{c \in C} X_{cuhd}\le k,\hspace{35pt} \forall u \in U, \forall d \in D
$$

In [17]:
daily_communication = mdl.add_constraints(
    (
        (mdl.sum(X_cuhd[(c,u,h,d)]  
               for c in range(0,MP.C) 
               for h in range(0,MP.H)) <= MP.k)
        for d in range(0,MP.D)
        for u in range(0,MP.U)))

##### - campaign communication limit(6)
$$
\sum_{d \in D}\sum_{h \in H} X_{cuhd}\le l_c,\hspace{35pt} \forall c \in C,\forall u \in U;
$$

In [18]:
campaign_communication = mdl.add_constraints(
    (
        (mdl.sum(X_cuhd[(c,u,h,d)]  
               for h in range(0,MP.H) 
               for d in range(0,MP.D)) <= MP.l_c[c] )
        for c in range(0,MP.C)
        for u in range(0,MP.U)))

##### - weekly quota(7)
$$
\sum_{d \in D}\sum_{h \in H}\sum_{c \in C}{X_{cuhd} q_{ic}}\le m_i,\hspace{35pt} \forall u \in U, \forall i \in I
$$

In [19]:
weekly_quota = mdl.add_constraints(
    (
        (mdl.sum(X_cuhd[(c,u,h,d)]*MP.q_ic[i,c]
               for c in range(0,MP.C)
               for h in range(0,MP.H) 
               for d in range(0,MP.D)) <= MP.m_i[i])
        for u in range(0,MP.U)
        for i in range(0,MP.I)))

##### - daily quota(8)
$$
\sum_{h \in H}\sum_{c \in C}{X_{cuhd} q_{ic}}\le n_i,\hspace{35pt} \forall u \in U,\, \forall d \in D, \forall i \in I
$$

In [20]:
daily_quota = mdl.add_constraints(
    (
        (mdl.sum(X_cuhd[(c,u,h,d)]*MP.q_ic[i,c]
               for c in range(0,MP.C) 
               for h in range(0,MP.H)) <= MP.n_i[i])
        for u in range(0,MP.U)
        for d in range(0,MP.D)
        for i in range(0,MP.I)))

##### Channel capacity (9)
$$
\sum_{c \in C}\sum_{u \in U}{X_{cuhd}}\le t_{hd},\hspace{35pt} \forall d \in D,\, \forall h \in H
$$

In [21]:
channel_capacity = mdl.add_constraints(
    (
        (mdl.sum(X_cuhd[(c,u,h,d)]
               for u in range(0,MP.U) 
               for c in range(0,MP.C)) <= MP.t_hd[h,d])
        for h in range(0,MP.H)
        for d in range(0,MP.D)))

### Solution

In [22]:
%%time

solution = mdl.solve(log_output=True, time_limit=600)
#solution = mdl.solve(log_output=True)

Version identifier: 20.1.0.0 | 2020-11-10 | 9bedb6d68
CPXPARAM_Read_DataCheck                          1
CPXPARAM_RandomSeed                              202001241
CPXPARAM_TimeLimit                               600
Parallel mode: deterministic, using up to 4 threads for concurrent optimization:
 * Starting dual Simplex on 1 thread...
 * Starting Barrier on 3 threads...
Tried aggregator 1 time.
LP Presolve eliminated 2593856 rows and 1054242 columns.
Reduced LP has 626165 rows, 1045758 columns, and 7283850 nonzeros.
Presolve time = 16.46 sec. (3264.91 ticks)
Symmetry aggregator did 1568872 additional substitutions.

Iteration log . . .
Iteration:     1   Dual objective     =      53987532.000000
Iteration:   528   Dual objective     =      10983317.000000
Iteration:  1117   Dual objective     =       9680725.000000
Iteration:  1742   Dual objective     =       7985579.000000
Perturbation started.
Iteration:  2403   Dual objective     =       7675974.000000
Iteration:  3028   Dual obje

In [23]:
solution.objective_value

4868048.0

In [24]:
solution.as_df().sum()["value"]

68097.0

In [25]:
display(len([val for ky,val in solution.as_name_dict().items() if val<0.5]))
display(len([val for ky,val in solution.as_name_dict().items() if val>0.5]))
display(len([val for ky,val in solution.as_name_dict().items() if val<1]))
display(len([val for ky,val in solution.as_name_dict().items() if val==0.5]))

0

68097

0

0

In [26]:
X_cuhd2 = np.zeros((MP.C,MP.U,MP.H,MP.D), dtype='int')
for ky,_ in solution.as_name_dict().items():
    exec(f'X_cuhd2{[int(i.split(":")[1]) for i in ky.split("_")[1:]]} = 1', {}, {'X_cuhd2':X_cuhd2})

c_i = 0
u_i = 1
h_i = 2
d_i = 3

mdl = model.Model([
    model.Constraint('eligibility',MP.eligibility, (c_i, u_i, h_i, d_i,)),
    model.Constraint('channel_capacity',MP.channel_capacity, (h_i, d_i,)),
    model.Constraint('daily_limitation',MP.daily_limitation, (u_i, d_i,)),
    model.Constraint('weekly_limitation',MP.weekly_limitation, (u_i,)),
    model.Constraint('campaign_limitation',MP.campaign_limitation, (c_i, u_i,)),
    model.Constraint('daily_quota',MP.daily_quota, (u_i, d_i,)),
    model.Constraint('one_channel',MP.one_channel, (c_i, u_i, d_i,)),
    model.Constraint('weekly_quota',MP.weekly_quota, (u_i,))
], MP.objective_fn)

def validate():
    for c in trange(MP.C, desc="Campaigns Loop"):
        for d in trange(MP.D, desc=f"Days Loop for campaign-{c}"):
            for h in range(MP.H):
                for u in range(MP.U):
                    if X_cuhd2[c,u,h,d]==1 and not mdl.execute(X_cuhd2, (c, u, h, d)):
                        #raise RuntimeError(f'{(c, u, h, d)} does not consistent with previous values!')
                        X_cuhd2[c,u,h,d]==0
    print("Solution is consistent with greedy")

validate()

model_value = mdl.calc_value(X_cuhd2)
model_comm_count = X_cuhd2.sum()

if (solution.objective_value == model_value):
    print(f"Solution value ({model_value}) is consistent with greedy")
else:
    print(f"Solution value ({model_value}) is not consistent with greedy")
    
if (solution.as_df().sum()["value"] == model_comm_count):
    print(f"Solution communication count ({model_comm_count}) is consistent with greedy")
else:
    print(f"Solution communication count ({model_comm_count}) is consistent with greedy")

Campaigns Loop:   0%|          | 0/10 [00:00<?, ?it/s]

Days Loop for campaign-0:   0%|          | 0/7 [00:00<?, ?it/s]

Days Loop for campaign-1:   0%|          | 0/7 [00:00<?, ?it/s]

Days Loop for campaign-2:   0%|          | 0/7 [00:00<?, ?it/s]

Days Loop for campaign-3:   0%|          | 0/7 [00:00<?, ?it/s]

...

Days Loop for campaign-6:   0%|          | 0/7 [00:00<?, ?it/s]

Days Loop for campaign-7:   0%|          | 0/7 [00:00<?, ?it/s]

Days Loop for campaign-8:   0%|          | 0/7 [00:00<?, ?it/s]

Days Loop for campaign-9:   0%|          | 0/7 [00:00<?, ?it/s]

Solution is consistent with greedy
Solution value (4868048) is consistent with greedy
Solution communication count (68097) is consistent with greedy
